Lee++ (NAR, 2016) says that IS*1* undertakes "local hopping".
Here, I aim to analyze whether the distribution of essential genes contribute to the local hopping.

# Overview

1. Load data
2. Identify esseitnal genes (using blastn)
3. Calculate average distance from the closest essential gene and compare with deletion length

### Input
Deletion boundary and corresponding IS IDs.
List of ranges of essential genes.

### Procedure
1. Identify new-existing type deletions (MA8, MA20).
2. Identify homologous sequences to the essential regions
	1. blast homologous genes
	2. remove the duplicates
	3. output closest essential genes for each IS 
3. Identify distances to the closest essential region.

# Loading

In [1]:
import os
import subprocess
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

## List of Essential genes
note that gene_df... is in the coordinates of MDS42_IS1.fa starting from thrC.

In [2]:
# Inputs
# get essential genes
essential_genes_file = os.path.join("..","misc","gene_df_ess_mds42_based_on_PEC.csv")
essential_genes = pd.read_csv(essential_genes_file)
essential_genes = essential_genes[["gene", "start", "end", "Essential_PEC"]]
essential_genes = essential_genes[essential_genes["Essential_PEC"] == True].\
			reset_index(drop=True)

# get MDS42 sequences
mds42_fasta = os.path.join("..","fasta","20230904","MDS42_IS1.fa")

essential_genes.head()

,gene,start,end,Essential_PEC
0,ribF,16232,17173,True
1,ileS,17216,20032,True
2,lspA,20032,20526,True
3,ispH,21102,22052,True
4,dapB,23199,24020,True


## IS positions

In [44]:
base_output_dir = os.path.join("exp", "multiple_runs11")
base_export = os.path.join(base_output_dir, "export")
data_export_dir = os.path.join("exp", "data", "ins_del_analysis")
file_name_base = "20240703_"
is_positions_df = os.path.join(base_export, "classify_IS_events", "IS_positions.csv")
is_positions_df = pd.read_csv(is_positions_df)
is_positions_df = is_positions_df.rename(columns = {"Gen":"gen_id", "Line":"line"})
is_positions_df.head()

,start,end,IS_strand,max_alignment_length,length,cluster_id,line,gen_id
0,476928,480020,reverse,3093,3093,0,L01-1,1
1,557167,560259,reverse,3093,3093,1,L01-1,1
2,1187831,1192502,forward,3093,4672,2,L01-1,1
3,1405123,1408215,forward,3093,3093,3,L01-1,1
4,1499230,1503892,reverse,3093,4663,4,L01-1,1


## Genome size

In [4]:
genome_sizes = os.path.join(base_export, "classify_IS_events", "genome_stats.csv")
genome_sizes = pd.read_csv(genome_sizes)
genome_sizes = genome_sizes[['Prefix', 'gen_id', 'genome_size', 'is_cnt']]
genome_sizes = genome_sizes.rename(columns = {"Prefix":"line"})
genome_sizes.head()

,line,gen_id,genome_size,is_cnt
0,L01-1,1,4003851,10
1,L01-2,1,4003851,10
2,L01-3,1,4003851,10
3,L01-4,1,4003851,10
4,L02-1,1,4026846,15


## Deletion length distributions

In [5]:
deletion_df = os.path.join(base_export, "classify_IS_events", "depth_interval.csv")
deletion_df = pd.read_csv(deletion_df)
deletion_df = deletion_df[['line', 'gen_id', 'group', 'start_fix', 'end_fix', 'is_start', 'is_end', 'depth']]
deletion_df = deletion_df.rename(columns = {"start_fix":"start", "end_fix":"end"})
deletion_df = deletion_df.query('depth == 0').drop(columns = ['depth'])
deletion_df.head()

,line,gen_id,group,start,end,is_start,is_end
1,L01-1,2,2,552531,560570,3,6
3,L01-1,2,4,2661331,2665538,21,22
6,L01-1,3,2,552045,552506,3,4
8,L01-1,3,4,1676236,1680732,18,21
10,L01-1,3,6,1791803,1792397,24,25


In [6]:
is_pos_in_ref_genome = os.path.join(base_export, "classify_IS_events", "IS_positions_in_ref_genome.csv")
is_pos_in_ref_genome = pd.read_csv(is_pos_in_ref_genome)
# Line, Gen, insert_id, genome, pos_id, IS_strand, position_status
is_pos_in_ref_genome = is_pos_in_ref_genome[['Gen', 'Line', 'insert_id', 'genome', 'pos_id', 'IS_strand', 'position_status']]
is_pos_in_ref_genome = is_pos_in_ref_genome.rename(columns = {"Gen":"gen_id", "Line":"line"})
is_pos_in_ref_genome = is_pos_in_ref_genome.query('genome == "Query"')
is_pos_in_ref_genome.query('position_status == "new"').head()

,gen_id,line,insert_id,genome,pos_id,IS_strand,position_status
22,2,L01-1,3,Query,1,False,new
23,2,L01-1,6,Query,1,False,new
24,2,L01-1,7,Query,2,False,new
25,2,L01-1,7,Query,2,False,new
27,2,L01-1,10,Query,3,False,new


# Identify essential genes

### Procedure
1. For each genome (line_id, gen_id)
2. Build blastdb and run blast with essential genes of MDS42 (PEC database) as query
3. Identify duplicated and missing essential genes

### Output:
- `merged_df`: Detailed alignment results including information about each significant match found during the BLAST searches.
- `summarised_df`: A summary report indicating the number of duplicated and missing essential genes for each line and genome analyzed.

## Misc functions

In [7]:
def create_blast_db(ref_fasta, db_dir, prefix, overwrite=False):
    """
    Create a BLAST database from a reference fasta file.
    :param ref_fasta: Path to the reference fasta file.
    :param db_dir: Directory where the BLAST database should be stored.
    :param prefix: Prefix to identify the database.
    """
    db_path = os.path.join(db_dir, prefix)
    command = ['makeblastdb', '-in', ref_fasta, '-dbtype', 'nucl', '-out', db_path]
    subprocess.run(command, check=True)
    return db_path

def run_blast(query_fasta, db_path, output_path):
    """
    Run BLASTn using a query fasta file against a BLAST database.
    :param query_fasta: Path to the query fasta file.
    :param db_path: Path to the BLAST database.
    :param output_path: Path to save the BLAST output.
    """
    command = [
        'blastn', '-query', query_fasta, '-db', db_path, 
        '-outfmt', '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen', 
        '-out', output_path
    ]
    subprocess.run(command, check=True)
    # add column names to the csv. currently data starts from the first row
    with open(output_path, 'r') as f:
        data = f.read()
    with open(output_path, 'w') as f:
        #tsv
        #f.write('qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen\n')
        f.write('qseqid\tsseqid\tpident\tlength\tmismatch\tgapopen\tqstart\tqend\tsstart\tsend\tevalue\tbitscore\tqlen\n')
        f.write(data)
    return output_path

def create_query_fasta(df, reference_fasta, output_fasta):
    """
    Create a query FASTA file from a pandas DataFrame containing start and end positions.
    :param df: DataFrame with columns ['Name', 'Start', 'End'].
    :param output_fasta: Path to the output FASTA file.
    """
    os.makedirs(os.path.dirname(output_fasta), exist_ok=True)
    records = []
    reference_seq = SeqIO.read(reference_fasta, "fasta").seq
    for index, row in df.iterrows():
    	sequence = reference_seq[row['start']:row['end']]
    	record = SeqRecord(Seq(sequence), id=row['gene'], description="")
    	records.append(record)
    SeqIO.write(records, output_fasta, "fasta")
    return output_fasta

def run_ess_blast(prefix, ref_fasta, query_fasta, tmp_dir):
	db_dir = os.path.join(tmp_dir, 'blastdb')
	output_dir = os.path.join(tmp_dir, 'blast-ess')
	result_csv = os.path.join(output_dir, f'{prefix}.blast-ess.csv')
	os.makedirs(db_dir, exist_ok=True)
	os.makedirs(output_dir, exist_ok=True)
	db_path = create_blast_db(ref_fasta, db_dir, prefix)
	run_blast(query_fasta, db_path, result_csv) # tsv
	return result_csv

## Main function

In [8]:
def identify_ess_genes(line_df, query_fasta, essential_genes, base_output_dir, tmp_dir):
	merged_df = pd.DataFrame()
	summarised_df = pd.DataFrame()
	for index, row in line_df.iterrows():
		line_id = row['line']
		gen_id = row['gen_id']

		res_csv_ = run_ess_blast(f'{line_id}.{gen_id}',\
			os.path.join(base_output_dir, "tmp", line_id, f'{line_id}_genome.{gen_id}.fasta'),\
			query_fasta, tmp_dir)
		res_ess_ = pd.read_csv(res_csv_, sep='\t')

		# filter partial matches
		threshold_length = 5
		threshold_ratio = 0.95
		threshold_evalue = 1e-10
		res_ess_ = res_ess_[(res_ess_['length'] > res_ess_['qlen'] - threshold_length) |\
			(res_ess_['length'] / res_ess_['qlen'] > threshold_ratio)]
		res_ess_ = res_ess_[(res_ess_['evalue'] < threshold_evalue)]

		res_ess_['line'] = line_id
		res_ess_['gen_id'] = gen_id
		# reorder by length
		res_ess_ = res_ess_.sort_values(by=['length'], ascending=False)
		res_ess_['duplicated'] = res_ess_.duplicated(subset=['qseqid'], keep=False)
		res_ess_ = res_ess_.rename(columns={'qseqid': 'gene'}).\
			drop(columns=['sseqid'])
		merged_df = pd.concat([merged_df, res_ess_], ignore_index=True)

		duplicated_genes = res_ess_[res_ess_['duplicated']]['gene'].unique()
		missing_genes = essential_genes[~essential_genes['gene'].isin(res_ess_['gene'])]
		summarised_df = pd.concat([summarised_df, pd.DataFrame({
			'line': [line_id],
			'gen_id': [gen_id],
			'duplicated_genes': [len(duplicated_genes)],
			'missing_genes': [len(missing_genes)], 
		})], ignore_index=True)

	#reindex 
	merged_df = merged_df.reset_index(drop=True)
	summarised_df = summarised_df.reset_index(drop=True)
	return merged_df, summarised_df

## Run

In [9]:
#L01-1, L01-2, ... L01-4, L02-1, ... L11-4
lines = ['L01', 'L02', 'L03', 'L04', 'L05', 'L06', 'L07', 'L08', 'L09', 'L10', 'L11']
#lines = ['L01', 'L02']
lines = [f'{line}-{i}' for line in lines for i in range(1, 5)]
gen_id = [1, 2, 3]

tmp_dir = os.path.join("tmp", "essential_gene_blast_identification")
query_fasta = os.path.join(tmp_dir, "mds42_ess.fasta")
create_query_fasta(essential_genes, mds42_fasta, query_fasta)

# lines for every gen-id
line_df = pd.DataFrame({'line': np.repeat(lines, len(gen_id)), 'gen_id': np.tile(gen_id, len(lines))})
merged_df, sumamrised_df = identify_ess_genes(line_df, query_fasta, essential_genes, base_output_dir, tmp_dir)



Building a new DB, current time: 07/06/2024 18:51:54
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L01-1.1
New DB title:  exp/multiple_runs11/tmp/L01-1/L01-1_genome.1.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L01-1.1
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 1 sequences in 0.022016 seconds.




Building a new DB, current time: 07/06/2024 18:51:54
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L01-1.2
New DB title:  exp/multiple_runs11/tmp/L01-1/L01-1_genome.2.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L01-1.2
Keep MBits: T
Maximum file size: 3000000000



Building a new DB, current time: 07/06/2024 18:51:56
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L02-2.2
New DB title:  exp/multiple_runs11/tmp/L02-2/L02-2_genome.2.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L02-2.2
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 1 sequences in 0.0220981 seconds.




Building a new DB, current time: 07/06/2024 18:51:56
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L02-2.3
New DB title:  exp/multiple_runs11/tmp/L02-2/L02-2_genome.3.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L02-2.3
Keep MBits: T
Maximum file size: 300000000



Building a new DB, current time: 07/06/2024 18:51:58
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L03-3.3
New DB title:  exp/multiple_runs11/tmp/L03-3/L03-3_genome.3.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L03-3.3
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 1 sequences in 0.021244 seconds.




Building a new DB, current time: 07/06/2024 18:51:58
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L03-4.1
New DB title:  exp/multiple_runs11/tmp/L03-4/L03-4_genome.1.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L03-4.1
Keep MBits: T
Maximum file size: 3000000000



Building a new DB, current time: 07/06/2024 18:51:59
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L05-1.1
New DB title:  exp/multiple_runs11/tmp/L05-1/L05-1_genome.1.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L05-1.1
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 1 sequences in 0.0216098 seconds.




Building a new DB, current time: 07/06/2024 18:52:00
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L05-1.2
New DB title:  exp/multiple_runs11/tmp/L05-1/L05-1_genome.2.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L05-1.2
Keep MBits: T
Maximum file size: 300000000



Building a new DB, current time: 07/06/2024 18:52:01
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L06-2.2
New DB title:  exp/multiple_runs11/tmp/L06-2/L06-2_genome.2.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L06-2.2
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 1 sequences in 0.0212419 seconds.




Building a new DB, current time: 07/06/2024 18:52:01
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L06-2.3
New DB title:  exp/multiple_runs11/tmp/L06-2/L06-2_genome.3.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L06-2.3
Keep MBits: T
Maximum file size: 300000000



Building a new DB, current time: 07/06/2024 18:52:03
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L07-3.3
New DB title:  exp/multiple_runs11/tmp/L07-3/L07-3_genome.3.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L07-3.3
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 1 sequences in 0.0219769 seconds.




Building a new DB, current time: 07/06/2024 18:52:03
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L07-4.1
New DB title:  exp/multiple_runs11/tmp/L07-4/L07-4_genome.1.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L07-4.1
Keep MBits: T
Maximum file size: 300000000



Building a new DB, current time: 07/06/2024 18:52:05
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L09-1.1
New DB title:  exp/multiple_runs11/tmp/L09-1/L09-1_genome.1.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L09-1.1
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 1 sequences in 0.0212591 seconds.




Building a new DB, current time: 07/06/2024 18:52:05
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L09-1.2
New DB title:  exp/multiple_runs11/tmp/L09-1/L09-1_genome.2.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L09-1.2
Keep MBits: T
Maximum file size: 300000000



Building a new DB, current time: 07/06/2024 18:52:06
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L10-2.2
New DB title:  exp/multiple_runs11/tmp/L10-2/L10-2_genome.2.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L10-2.2
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 1 sequences in 0.0213559 seconds.




Building a new DB, current time: 07/06/2024 18:52:06
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L10-2.3
New DB title:  exp/multiple_runs11/tmp/L10-2/L10-2_genome.3.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L10-2.3
Keep MBits: T
Maximum file size: 300000000



Building a new DB, current time: 07/06/2024 18:52:08
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L11-3.3
New DB title:  exp/multiple_runs11/tmp/L11-3/L11-3_genome.3.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L11-3.3
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 1 sequences in 0.0210679 seconds.




Building a new DB, current time: 07/06/2024 18:52:08
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L11-4.1
New DB title:  exp/multiple_runs11/tmp/L11-4/L11-4_genome.1.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L11-4.1
Keep MBits: T
Maximum file size: 300000000

## Result (no missing)

In [10]:
# all essential genes are found in the list
sumamrised_df.query("duplicated_genes > 0 | missing_genes > 0")

,line,gen_id,duplicated_genes,missing_genes
4,L01-2,2,2,0
5,L01-2,3,2,0
14,L02-1,3,13,0
16,L02-2,2,2,0
17,L02-2,3,14,0
20,L02-3,3,9,0
23,L02-4,3,3,0
29,L03-2,3,7,0
35,L03-4,3,3,0
37,L04-1,2,4,0


## Test run (L04-3)

In [11]:
# L04-3 should have duplicated essential genes. I use this to test the code
line_id = 'L04-3'
gen_id = 2
res_csv_ = run_ess_blast(f'{line_id}.{gen_id}',\
	os.path.join(base_output_dir, "tmp", line_id, f'{line_id}_genome.{gen_id}.fasta'),\
	query_fasta, tmp_dir)

res_ess_ = pd.read_csv(res_csv_, sep='\t')
res_ess_.head()



Building a new DB, current time: 07/06/2024 18:52:09
New DB name:   /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L04-3.2
New DB title:  exp/multiple_runs11/tmp/L04-3/L04-3_genome.2.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kanai/documents/analysis/myTELR/syri/ipynb/tmp/essential_gene_blast_identification/blastdb/L04-3.2
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 1 sequences in 0.022994 seconds.




,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen
0,ribF,"edge_3-,edge_6+,edge_2-,edge_4+,edge_5-,edge_4...",100.0,941,0,0,1,941,575672,576612,0.0,1738,941
1,ileS,"edge_3-,edge_6+,edge_2-,edge_4+,edge_5-,edge_4...",100.0,2816,0,0,1,2816,576656,579471,0.0,5201,2816
2,lspA,"edge_3-,edge_6+,edge_2-,edge_4+,edge_5-,edge_4...",100.0,494,0,0,1,494,579472,579965,0.0,913,494
3,ispH,"edge_3-,edge_6+,edge_2-,edge_4+,edge_5-,edge_4...",100.0,950,0,0,1,950,580542,581491,0.0,1755,950
4,dapB,"edge_3-,edge_6+,edge_2-,edge_4+,edge_5-,edge_4...",100.0,821,0,0,1,821,582639,583459,0.0,1517,821


In [12]:
res_ess_['is_duplicated'] = res_ess_.duplicated(subset=['qseqid'], keep=False)
unique_hits = res_ess_[~res_ess_['is_duplicated']]

duplicated_genes = res_ess_[res_ess_['is_duplicated']]['qseqid'].unique()
missing_genes = essential_genes[~essential_genes['gene'].isin(res_ess_['qseqid'])]

print(f"Number of unique hits: {len(unique_hits)}")
print(f"Number of missed essential genes: {len(missing_genes)}")
print(f"Number of duplicated hits: {len(duplicated_genes)}")

Number of unique hits: 292
Number of missed essential genes: 0
Number of duplicated hits: 10


# Calculate average distance from the closest essential gene

For a region to be deleted by NE type IS deletion(recombination between Newly inserted IS with a preExisting one), which was the majority in Lee 2016,
the region must be non-essential.

Here, I identify if the local hopping is due to the distribution of essential genes
or due to an innate property of the IS.

In [13]:
# by chatgpt
import pandas as pd

# Function to find the closest essential gene and IS end
def find_closest_gene(is_row, genes_df):
    # Initialize minimum distance and details for the closest approach
    min_distance = float('inf')
    closest_gene = None
    gene_end = None
    is_end_involved = None
    
    for _, gene_row in genes_df.iterrows():
        # Distances from IS start and end to both ends of the gene
        distances = {
            ('5_prime', 'start'): abs(is_row['start'] - gene_row['sstart']),
            ('5_prime', 'end'): abs(is_row['end'] - gene_row['sstart']),
            ('3_prime', 'start'): abs(is_row['start'] - gene_row['send']),
            ('3_prime', 'end'): abs(is_row['end'] - gene_row['send'])
        }
        
        # Identify the minimum distance and corresponding details
        for (gene_side, is_side), distance in distances.items():
            if distance < min_distance:
                min_distance = distance
                closest_gene = gene_row['gene']
                gene_end = gene_side
                is_end_involved = is_side
                position = gene_row['sstart'] if gene_side == '5_prime' else gene_row['send']
                
    # Return details including which IS end (start or end) is closest to the gene
    return pd.Series([closest_gene, gene_end, is_end_involved, min_distance, position],\
			 index=['closest_gene', 'gene_end', 'IS_end', 'distance', 'position'])

In [14]:
# test
is_df = is_positions_df.query("line == 'L04-3' and gen_id == 2")
essential_genes_df = merged_df.query("line == 'L04-3' and gen_id == 2")

# Apply the function to each IS row
#is_df[['closest_gene', 'gene_end', 'is_end','distance']] = is_df.apply(find_closest_gene, genes_df=essential_genes_df, axis=1)
# use loc
is_df.loc[:, ['closest_gene', 'gene_end', 'IS_end','distance', 'position']] = \
	is_df.apply(find_closest_gene, genes_df=essential_genes_df, axis=1)
is_df.head()

,start,end,IS_strand,max_alignment_length,length,cluster_id,line,gen_id,closest_gene,gene_end,IS_end,distance,position
831,42391,45482,reverse,3093,3092,0,L04-3,2,rho,3_prime,start,466,41925
832,121558,124650,forward,3093,3093,1,L04-3,2,polA,5_prime,end,2763,127413
833,355703,358793,reverse,3092,3091,2,L04-3,2,ssb,3_prime,start,608,355095
834,444530,447621,reverse,3092,3092,3,L04-3,2,ppa,5_prime,end,10760,458381
835,552106,555198,reverse,3093,3093,4,L04-3,2,ribF,5_prime,end,20474,575672


In [15]:
essential_genes_df

,gene,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,line,gen_id,duplicated
13048,mukB,100.0,4460,0,0,1,4460,1418203,1422662,0.000000e+00,8237,4460,L04-3,2,False
13049,rpoC,100.0,4223,0,0,1,4223,265789,270011,0.000000e+00,7799,4223,L04-3,2,False
13050,rpoB,100.0,4028,0,0,1,4028,261684,265711,0.000000e+00,7439,4028,L04-3,2,False
13051,ftsK,100.0,3989,0,0,1,3989,1929228,1925240,0.000000e+00,7367,3989,L04-3,2,True
13052,ftsK,100.0,3989,0,0,1,3989,1372000,1375988,0.000000e+00,7367,3989,L04-3,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13355,hisR,100.0,76,0,0,1,76,59860,59935,6.390000e-35,141,76,L04-3,2,False
13356,trpT,100.0,75,0,0,1,75,21211,21285,2.260000e-34,139,75,L04-3,2,False
13357,thrU,100.0,75,0,0,1,75,255827,255901,2.260000e-34,139,75,L04-3,2,False
13358,glyT,100.0,74,0,0,1,74,256112,256185,7.970000e-34,137,74,L04-3,2,False


In [16]:
import pandas as pd

# Example dataframes
# Assuming these are already defined: is_df and essential_genes_df

# Additional input
genome_size = 5000000  # Example genome size, adjust as necessary

def calculate_circular_distance(start, end, genome_size):
    """
    Calculate the shortest distance between two points in a circular genome.
    """
    direct_distance = abs(end - start)
    return min(direct_distance, genome_size - direct_distance)

def find_closest_genes_around_is(is_row, genes_df, genome_size):
    """
    Find the closest essential genes to both the 5' and 3' ends of an IS, considering circular genome.
    """
    # Initialize variables to track the closest genes and distances
    closest_gene_5_prime = None
    closest_gene_3_prime = None
    min_distance_5_prime = float('inf')
    min_distance_3_prime = float('inf')
    position_5_prime = None
    position_3_prime = None
    
    # Loop through each essential gene to find the closest to 5' and 3' ends of IS
    for _, gene_row in genes_df.iterrows():
        # For 5' end, consider genes at lower indexes (considering circularity)
        if gene_row['send'] < is_row['start']:
            distance_5_prime = calculate_circular_distance(gene_row['send'], is_row['start'], genome_size)
            if distance_5_prime < min_distance_5_prime:
                min_distance_5_prime = distance_5_prime
                closest_gene_5_prime = gene_row['gene']
                position_5_prime = gene_row['send']
        # For 3' end, consider genes at higher indexes (considering circularity)
        elif gene_row['sstart'] > is_row['end']:
            distance_3_prime = calculate_circular_distance(is_row['end'], gene_row['sstart'], genome_size)
            if distance_3_prime < min_distance_3_prime:
                min_distance_3_prime = distance_3_prime
                closest_gene_3_prime = gene_row['gene']
                position_3_prime = gene_row['sstart']
    
    return pd.Series([closest_gene_5_prime, position_5_prime, min_distance_5_prime, closest_gene_3_prime, position_3_prime, min_distance_3_prime],\
                     index=['closest_gene_5_prime', 'position_5_prime', 'distance_5_prime', 'closest_gene_3_prime', 'position_3_prime', 'distance_3_prime'])

In [17]:
# test I manually checked the first two is right.
is_df = is_positions_df.query("line == 'L04-3' and gen_id == 2")
essential_genes_df = merged_df.query("line == 'L04-3' and gen_id == 2 and not duplicated")[['gene', 'sstart', 'send']]
essential_genes_df = pd.concat([essential_genes_df,\
		pd.DataFrame({'gene': ['oriC', 'oriC'], 'sstart': [1, genome_size], 'send': [1, genome_size]})], ignore_index=True)

#is_df[['closest_gene_5_prime', 'position_5_prime', 'distance_5_prime', 'closest_gene_3_prime', 'position_3_prime', 'distance_3_prime']] = is_df.apply(find_closest_genes_around_is, genes_df=essential_genes_df, genome_size=genome_size, axis=1)
# use loc
is_df.loc[:, ['closest_gene_5_prime', 'position_5_prime', 'distance_5_prime', 'closest_gene_3_prime', 'position_3_prime', 'distance_3_prime']] = \
	is_df.apply(find_closest_genes_around_is, genes_df=essential_genes_df, genome_size=genome_size, axis=1)
is_df.head()

,start,end,IS_strand,max_alignment_length,length,cluster_id,line,gen_id,closest_gene_5_prime,position_5_prime,distance_5_prime,closest_gene_3_prime,position_3_prime,distance_3_prime
831,42391,45482,reverse,3093,3092,0,L04-3,2,rho,41925,466,argX,59726,14244
832,121558,124650,forward,3093,3093,1,L04-3,2,hemG,112499,9059,polA,127413,2763
833,355703,358793,reverse,3092,3091,2,L04-3,2,ssb,355095,608,groS,377254,18461
834,444530,447621,reverse,3092,3092,3,L04-3,2,rpsR,432628,11902,ppa,458381,10760
835,552106,555198,reverse,3093,3093,4,L04-3,2,dnaT,516211,35895,ribF,575672,20474


In [18]:
# genome size should not matter much as deletions can't cover oriC
def find_closest_genes_around_is_batch(line_df, genome_sizes, merged_essential_genes, is_positions_df):
	output_df = pd.DataFrame()
	for _, row in line_df.iterrows():
		line_id = row['line']
		gen_id = row['gen_id']
		genome_size = genome_sizes.query(f"line == '{line_id}' and gen_id == {gen_id}")['genome_size'].values[0]

		essential_genes_df = merged_essential_genes.query(f"line == '{line_id}' and gen_id == {gen_id} and not duplicated")[['gene', 'sstart', 'send']]
		essential_genes_df = pd.concat([essential_genes_df,\
			pd.DataFrame({'gene': ['oriC', 'oriC'], 'sstart': [1, genome_size], 'send': [1, genome_size]})], ignore_index=True)

		is_df = is_positions_df.query(f"line == '{line_id}' and gen_id == {gen_id}")
		is_df.loc[:, ['closest_gene_5_prime', 'position_5_prime', 'distance_5_prime', 'closest_gene_3_prime', 'position_3_prime', 'distance_3_prime']] = \
			is_df.apply(find_closest_genes_around_is, genes_df=essential_genes_df, genome_size=genome_size, axis=1)

		output_df = pd.concat([output_df, is_df], ignore_index=True)
	return output_df

In [19]:
is_distance_df = find_closest_genes_around_is_batch(line_df, genome_sizes, merged_df, is_positions_df)
is_distance_df.head()

,start,end,IS_strand,max_alignment_length,length,cluster_id,line,gen_id,closest_gene_5_prime,position_5_prime,distance_5_prime,closest_gene_3_prime,position_3_prime,distance_3_prime
0,476928,480020,reverse,3093,3093,0,L01-1,1,yjeE,470186,6742,rpsR,503080,23060
1,557167,560259,reverse,3093,3093,1,L01-1,1,ppa,526892,30275,valS,561324,1065
2,1187831,1192502,forward,3093,4672,2,L01-1,1,leuS,1175771,12060,asnS,1205042,12540
3,1405123,1408215,forward,3093,3093,3,L01-1,1,infA,1266184,138939,fldA,1476321,68106
4,1499230,1503892,reverse,3093,4663,4,L01-1,1,lnt,1496376,2854,fabA,1519050,15158


## Analyse results
group by {line, gen_id} and calculate average, median merged {distance_5_prime, distance_3_prime}
Merging the 5' and 3' prime distances into one column

In [20]:
merged_distances = pd.melt(is_distance_df, id_vars=['line', 'gen_id'], value_vars=['distance_5_prime', 'distance_3_prime'], 
                           var_name='prime_end', value_name='distance')

# Group by line and gen_id, then calculate average and median
grouped = merged_distances.groupby(['line', 'gen_id'])
avg_distance = grouped['distance'].mean().reset_index(name='average_distance')
median_distance = grouped['distance'].median().reset_index(name='median_distance')

# Calculating expected value
expected_value = grouped.apply(lambda x: x['distance'].pow(2).sum() / x['distance'].sum() / 2).reset_index(name='expected')

# Merging the results
is_distance_stats = avg_distance.merge(median_distance, on=['line', 'gen_id']).merge(expected_value, on=['line', 'gen_id'])
is_distance_stats.head(10)

,line,gen_id,average_distance,median_distance,expected
0,L01-1,1,37956.400000,14958.0,50612.642789
1,L01-1,2,38750.375000,14957.5,49624.484261
2,L01-1,3,30962.933333,18428.5,32979.744036
3,L01-2,1,37956.400000,14958.0,50612.642789
4,L01-2,2,32031.406250,14956.5,43147.040845
5,L01-2,3,35587.261905,14958.0,47751.338572
6,L01-3,1,37956.400000,14958.0,50612.642789
7,L01-3,2,42077.833333,21731.5,53974.845497
8,L01-3,3,35247.043478,19606.5,48134.267662
9,L01-4,1,37956.400000,14958.0,50612.642789


## median of expected distance  (not used)

In [21]:
# median of expected distances
expected_distances = is_distance_stats.query('gen_id == 1 | gen_id == 2').\
	query('not(line == "L04-1" and gen_id == 2)').expected
print(np.percentile(is_distance_stats.query('gen_id == 1 | gen_id == 2').expected, 75) - np.percentile(is_distance_stats.query('gen_id == 1 | gen_id == 2').expected, 25))
# in text
print(f'The median (IQR) of the expected distances between IS elements and the closest essential genes is {np.median(expected_distances):.2f} ({np.percentile(expected_distances, 25):.2f}-{np.percentile(expected_distances, 75):.2f}).')
print(f'Number of genomes: {len(expected_distances)}')

10869.794110541989
The median (IQR) of the expected distances between IS elements and the closest essential genes is 40831.26 (37021.08-47950.56).
Number of genomes: 87


## median of expected median distance (not used)

In [22]:
# calculate the expected median
def calculate_expected_median(group):
    # Sort the group by distance
    sorted_group = group.sort_values(by='distance')
    
    # Calculate the cumulative sum of distances
    cumulative_sum = sorted_group['distance'].cumsum()
    
    # Total sum of distances for the group
    total_sum = cumulative_sum.iloc[-1]
    
    # Find the median value: where the cumulative sum reaches half of the total sum
    # At such point, half the events will be related to ISs with shorter distances and the other half with longer distances
    median_index = cumulative_sum[cumulative_sum >= total_sum / 2].index[0]
    weighted_median = sorted_group.loc[median_index, 'distance'] / 2
    
    return weighted_median

# Apply the weighted median calculation to each group
median_values = grouped.apply(calculate_expected_median).reset_index(name='weighted_median')

# Merging the weighted median with the previous results
final_results = is_distance_stats.merge(median_values, on=['line', 'gen_id'])

final_results.head()

,line,gen_id,average_distance,median_distance,expected,weighted_median
0,L01-1,1,37956.400000,14958.0,50612.642789,48877.0
1,L01-1,2,38750.375000,14957.5,49624.484261,48875.5
2,L01-1,3,30962.933333,18428.5,32979.744036,34049.5
3,L01-2,1,37956.400000,14958.0,50612.642789,48877.0
4,L01-2,2,32031.406250,14956.5,43147.040845,34051.0


In [23]:
# IQR of weighed medians
expected_distances = final_results.query('gen_id == 1 | gen_id == 2').\
	query('not(line == "L04-1" and gen_id == 2)').weighted_median
# in text
print(f'The median (IQR) of the expected distances between IS elements and the closest essential genes is:')
print(f'{np.median(expected_distances):.2f} ({np.percentile(expected_distances, 25):.2f}-{np.percentile(expected_distances, 75):.2f}).')
print(f'Number of genomes: {len(expected_distances)}')

The median (IQR) of the expected distances between IS elements and the closest essential genes is:
33787.50 (32818.50-47095.25).
Number of genomes: 87


## Expected median deletions by normalizing the weights for each genome

In [33]:
merged_distances_median = merged_distances.query('gen_id == 1 | gen_id == 2').\
	query('not(line == "L04-1" and gen_id == 2)').\
	sort_values(by='distance')

In [34]:
# Calculate initial weights based on distance
merged_distances_median['initial_weight'] = merged_distances_median['distance']

# Determine factors based on gen_id (8 for gen_id = 1, 12 for gen_id = 2)
merged_distances_median['factor'] = np.where(merged_distances_median['gen_id'] == 1, 8, 12)

# Group by 'line' and 'gen_id', and calculate the sum of weights for normalization
grouped_weights = merged_distances_median.groupby(['line', 'gen_id'])['initial_weight'].transform('sum')

# Normalize weights within each group and apply factor
merged_distances_median['normalized_weight'] = (merged_distances_median['initial_weight'] / grouped_weights) * merged_distances_median['factor']

# Print DataFrame to see the normalized weights
print(merged_distances_median)

       line  gen_id         prime_end  distance  initial_weight  factor  \
1941  L09-4       1  distance_5_prime        23              23       8   
1953  L09-4       2  distance_5_prime        23              23      12   
1891  L09-3       1  distance_5_prime        23              23       8   
1865  L09-2       2  distance_5_prime        23              23      12   
1905  L09-3       2  distance_5_prime        23              23      12   
...     ...     ...               ...       ...             ...     ...   
2084  L10-3       2  distance_5_prime    239737          239737      12   
2128  L10-4       2  distance_5_prime    239740          239740      12   
1990  L10-1       2  distance_5_prime    239742          239742      12   
843   L04-3       2  distance_5_prime    340675          340675      12   
844   L04-3       2  distance_5_prime    345583          345583      12   

      normalized_weight  
1941           0.000272  
1953           0.000361  
1891           0.0002

## STATS: weighed median

In [35]:
# Calculate cumulative weight to find the median
# This approach assumes uniform distribution of deletion sizes from 1 to distance
total_weight = merged_distances_median['normalized_weight'].sum()
merged_distances_median['cumulative_weight'] = merged_distances_median['normalized_weight'].cumsum()

# Locate the median by finding where the cumulative weight exceeds half of the total weight
half_total_weight = total_weight / 2
median_row = merged_distances_median[merged_distances_median['cumulative_weight'] >= half_total_weight].iloc[0]
median_deletion_size = median_row['distance'] / 2 + 20  # Median of uniform distribution from 1 to distance

# Display the analytically estimated expected median deletion size
print("Analytically estimated expected median deletion size:", median_deletion_size)

Analytically estimated expected median deletion size: 34338.5


In [36]:
merged_distances_median

,line,gen_id,prime_end,distance,initial_weight,factor,normalized_weight,cumulative_weight
1941,L09-4,1,distance_5_prime,23,23,8,0.000272,0.000272
1953,L09-4,2,distance_5_prime,23,23,12,0.000361,0.000634
1891,L09-3,1,distance_5_prime,23,23,8,0.000250,0.000883
1865,L09-2,2,distance_5_prime,23,23,12,0.000336,0.001220
1905,L09-3,2,distance_5_prime,23,23,12,0.000230,0.001449
...,...,...,...,...,...,...,...,...
2084,L10-3,2,distance_5_prime,239737,239737,12,2.046176,860.820251
2128,L10-4,2,distance_5_prime,239740,239740,12,2.311581,863.131831
1990,L10-1,2,distance_5_prime,239742,239742,12,2.411804,865.543636
843,L04-3,2,distance_5_prime,340675,340675,12,1.219398,866.763034


#### Bootstrap with percentile method (not used)

In [37]:
# Function to calculate the weighted median for a given DataFrame
def weighted_median(df, value_column, weight_column):
    df_sorted = df.sort_values(value_column)
    cumsum = df_sorted[weight_column].cumsum()
    cutoff = df_sorted[weight_column].sum() / 2.0
    return df_sorted[cumsum >= cutoff][value_column].iloc[0]/2 + 20

# Bootstrapping function to resample and calculate the weighted median
def bootstrap_sample(df, n_iterations=1000):
    results = []
    for _ in range(n_iterations):
        # Sample with replacement
        sample = df.sample(n=len(df), replace=True)
        # Calculate the weighted median of 'distance' using 'normalized_weight'
        median = weighted_median(sample, 'distance', 'normalized_weight')
        results.append(median)
    return np.percentile(results, [2.5, 97.5])  # 95% confidence interval

# Perform bootstrapping
confidence_interval = bootstrap_sample(merged_distances_median)

print("95% Confidence Interval for the Weighted Median of Distance:", confidence_interval)

95% Confidence Interval for the Weighted Median of Distance: [33807. 37755.]


#### STATS: Bootstrap with BCa

In [38]:
import numpy as np
import pandas as pd
from scipy.stats import bootstrap

# Function to calculate the weighted median
def weighted_median(data, weights):
    # Ensure data and weights are numpy arrays for efficient computation
    data, weights = np.array(data), np.array(weights)
    # Sort data and corresponding weights
    sorted_indices = np.argsort(data)
    sorted_data = data[sorted_indices]
    sorted_weights = weights[sorted_indices]
    # Compute cumulative weights
    cumsum_weights = np.cumsum(sorted_weights)
    # Find the index where the cumulative weight is greater than or equal to half the total weight
    cutoff = cumsum_weights[-1] / 2.0 # -1 returns total weight
    median_index = np.searchsorted(cumsum_weights, cutoff) # a[i-1] < v <= a[i]
    return sorted_data[median_index] / 2.0 + 20  # given the distance the expected median is halft + 1 for unifromal distribution

# Function to perform bootstrapping using the BCa method
def bootstrap_bca_median(df, value_column, weight_column, n_resamples=1000):
    # Extract values and weights from DataFrame
    values = df[value_column].to_numpy()
    weights = df[weight_column].to_numpy()

    # Define a function for use with bootstrap that computes the weighted median
    def compute_median(indices):
        # indices to sample data and weights for the bootstrap sample
        sampled_values = values[indices]
        sampled_weights = weights[indices]
        return weighted_median(sampled_values, sampled_weights)

    # Perform bootstrap with BCa method
    res = bootstrap((np.arange(len(values)),), statistic=compute_median, 
                    n_resamples=n_resamples, method='BCa', vectorized=False)
    return res.confidence_interval

# Perform bootstrapping and print the confidence interval
confidence_interval = bootstrap_bca_median(merged_distances_median, 'distance', 'normalized_weight')

print("BCa Confidence Interval for the Weighted Median of Distance:", confidence_interval)

BCa Confidence Interval for the Weighted Median of Distance: ConfidenceInterval(low=33807.0, high=38628.0)


### Save weight data

In [45]:
merged_distances_median.to_csv(os.path.join(data_export_dir, file_name_base + "essential_gene_distance_weighted.csv"))